In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasetfornina/ninaprodb1train.pkl
/kaggle/input/datasetfornina/ninaprodb1test.pkl


In [19]:
import numpy as np
import os
import random
#from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import math

In [20]:
import pickle
# Load the dataset from pickle file
with open('/kaggle/input/datasetfornina/ninaprodb1test.pkl', 'rb') as f:
    test_data = pickle.load(f)
with open('/kaggle/input/datasetfornina/ninaprodb1train.pkl', 'rb') as f:
    train_data = pickle.load(f)

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
from tensorflow.keras import layers, Sequential, optimizers, Input, Model

**WORKED CODE HERE**

In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import gc
import random
import os

import matplotlib.pyplot as plt
import json

In [24]:
epochs = 64
learning_rate = 1e-3
batch_size = 16
method = "default"
dataset_type = 1

In [25]:
### import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical


class Nina1Dataset(tf.keras.utils.Sequence):
    def __init__(self, dataframe, batch_size):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.scaler = StandardScaler()
        self.scaler.fit(np.concatenate(self.dataframe['emg'].tolist()))
        
    def __len__(self):
        return int(np.ceil(len(self.dataframe) / self.batch_size))

    def __getitem__(self, idx):
        batch_data = self.dataframe[idx * self.batch_size:(idx + 1) * self.batch_size]

        batch_input_data = []
        batch_labels = []

        for i, target_row in batch_data.iterrows():
            data = target_row['emg'][:500]

            # Zero-Padding
            if len(data) < 500:
                data = np.concatenate((data, np.zeros((500 - len(data), 10))), axis=0)
            data = self.scaler.transform(data)

            # Division data by time-segment
            #input_data = np.transpose(data.reshape((25, 20, 10)), (0, 2, 1))
            input_data = data.reshape((25, 20, 10))
            label = target_row['stimulus']
            #label = to_categorical(label)
            batch_input_data.append(input_data)
            batch_labels.append(label)

        # Check if the batch size is smaller than the desired batch_size
        if len(batch_data) < self.batch_size:
            # Create a dummy batch with all elements set to zero
            dummy_input_data = np.zeros((self.batch_size,) + input_data.shape, dtype=np.float32)
            dummy_labels = np.zeros((self.batch_size,), dtype=np.int32)
            dummy_input_data[:len(batch_input_data)] = np.array(batch_input_data)
            dummy_labels[:len(batch_labels)] = np.array(batch_labels)
            dummy_labels = to_categorical(dummy_labels,num_classes=52)  # Convert labels to one-hot encoding
            
            return dummy_input_data, dummy_labels
        
        batch_labels = to_categorical(batch_labels,num_classes=52)  # Convert labels to one-hot encoding
        
        return np.array(batch_input_data), np.array(batch_labels)





# Parameters
batch_size = 16
train_dir = '/kaggle/input/datasetfornina/ninaprodb1train.pkl'
test_dir = '/kaggle/input/datasetfornina/ninaprodb1test.pkl'

# Set up dataset

train = pd.read_pickle(train_dir)
eval_data = pd.read_pickle(test_dir)

# Load train data
train_data = pd.read_pickle(train_dir)

# Split data into train and validation sets
train_data, val_data = train_test_split(train_data, test_size=0.3, random_state=21)

train_data = train_data.sample(frac=1).reset_index(drop=True)
val_data = val_data.sample(frac=1).reset_index(drop=True)
eval_data = eval_data.sample(frac=1).reset_index(drop=True)

# Create train, test and validation datasets
train_dataset = Nina1Dataset(train_data, batch_size=batch_size)
val_dataset= Nina1Dataset(val_data, batch_size=batch_size)
test_dataset= Nina1Dataset(eval_data, batch_size=batch_size)


print(train_dataset)

In [26]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
# Define the CNN-BiLSTM model
from tensorflow.keras.layers import LSTM, Bidirectional, Dropout
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, ReLU
from tensorflow.keras.activations import tanh

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.layers import TimeDistributed

def cnn(x):
    #print(x.shape)
    #x = Reshape((25, 20,10))(x)  
    print(x.shape)
    x = TimeDistributed((Conv1D(64, kernel_size=9, strides=2, padding='same', activation=tanh)))(x)
    print(x.shape)
    #(Batch, 10, 64)

    x = TimeDistributed(MaxPooling1D(pool_size=8, strides=2))(x)
    print(x.shape)
    #(Batch, 2, 64)
    
    x = TimeDistributed((Conv1D(64, kernel_size=5, strides=2, padding='same', activation=tanh)))(x)
    #(Batch, 1, 64)
    x = TimeDistributed(BatchNormalization(epsilon=1e-6, momentum=0.95))(x)
    
    x = TimeDistributed((Conv1D(64, kernel_size=5, strides=2, padding='same', activation=tanh)))(x)
    print(x.shape)
    #(Batch, 1, 64)
    #x = TimeDistributed(Dropout(0.2093))(x)
    x = TimeDistributed(BatchNormalization(epsilon=1e-6, momentum=0.95))(x)
    
    #x = TimeDistributed(ReLU())(Conv1D(64, kernel_size=3, strides=2, padding='same')(x))
    x = TimeDistributed((Conv1D(64, kernel_size=3, strides=2, padding='same', activation=tanh)))(x)
    #print(x.shape)
    #(Batch, 1, 64)
    #x = TimeDistributed(Dropout(0.2093))(x)
    #x = TimeDistributed(BatchNormalization(epsilon=1e-6, momentum=0.95))(x)
    x = TimeDistributed(Flatten())(x)
    print(x.shape)
    # (Batch, 64)

    return x

def Bi_LSTMModel(input_shape,x):
    #model = Sequential()
    # Hidden dimensions
    hidden_dim = 200
    print(x.shape)
    print(11)
    x = Bidirectional(LSTM(hidden_dim, return_sequences=True, dropout=0.2093), input_shape=input_shape)(x)
    #x = Dropout(0.2093)(x)
    print(x.shape)
    print(11)
    x = Bidirectional(LSTM(hidden_dim, return_sequences=True, dropout=0.2093))(x)

    print(x.shape)
    print(11)
    #x = Dropout(0.2093)(x)
    x = Flatten()(x)
    # ( ,10000)

    return x
def EMGHandNet(input_shape, num_classes):
    # Define the input layer
    x = Input(shape=input_shape)
    inputs = x
    #print(x.shape)
    #(batch, 25, 20, 10)
    #temp = [cnn(x[:, t, :, :]) for t in range(x.shape[1])]
    #x = tf.stack(temp, axis=1)
    #print(x.shape)
    x = cnn(x)
   
    #print(x.shape)
    x = Bi_LSTMModel(x.shape[1:],x)
    #print(x.shape)
    #x = Dropout(0.2093)(x)
    
    x = Dense(512, activation='tanh')(x)
    #print(x.shape)
    x = Dropout(0.2093)(x)

    # Add the output layer
    output_layer = Dense(52, activation='softmax')(x)
    # Create the model
    model = Model(inputs=inputs, outputs=output_layer)

    return model


num_classes = 52  # Adjust based on the number of hand activity classes


# Create an instance of the EMGHandNet model
model = EMGHandNet((25,20,10), 52)


# Compile the model

#dtype = tf.float32

initial_learning_rate = 0.001
decay_steps = 1000
decay_rate = 0.9
batch_size = 16
# Define your model and its optimizer
# Define learning rate schedule
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps, decay_rate)

    # Compile the model with learning rate schedule
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])
history = model.fit(train_dataset,
                    epochs=200,
                    validation_data=val_dataset,
                    batch_size=16
                    )

(None, 25, 20, 10)
(None, 25, 10, 64)
(None, 25, 2, 64)
(None, 25, 1, 64)
(None, 25, 64)
(None, 25, 64)
11
(None, 25, 400)
11
(None, 25, 400)
11
Epoch 1/200
430/430 [==============================] - 25s 34ms/step - loss: 3.1164 - accuracy: 0.2096 - val_loss: 2.4540 - val_accuracy: 0.3135
Epoch 2/200
430/430 [==============================] - 12s 29ms/step - loss: 2.2118 - accuracy: 0.3842 - val_loss: 1.9764 - val_accuracy: 0.4311
Epoch 3/200
430/430 [==============================] - 13s 31ms/step - loss: 1.7517 - accuracy: 0.5004 - val_loss: 1.7334 - val_accuracy: 0.4885
Epoch 4/200
430/430 [==============================] - 12s 29ms/step - loss: 1.3140 - accuracy: 0.6096 - val_loss: 1.4746 - val_accuracy: 0.5699
Epoch 5/200
430/430 [==============================] - 13s 30ms/step - loss: 0.9110 - accuracy: 0.7183 - val_loss: 1.2516 - val_accuracy: 0.6291
Epoch 6/200
430/430 [==============================] - 13s 30ms/step - loss: 0.6276 - accuracy: 0.8000 - val_loss: 1.1325 - val_ac

In [28]:
true_labels = []  # True labels for the test data
predicted_labels = []  # Predicted labels for the test data

for batch_data, batch_labels in test_dataset:
    batch_predictions = model.predict(batch_data)
    batch_predicted_labels = np.argmax(batch_predictions, axis=1)
    predicted_labels.extend(batch_predicted_labels)
    true_labels.extend(np.argmax(batch_labels, axis=1))

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

accuracy = np.mean(true_labels == predicted_labels)

print(accuracy)

1/1 [==============================] - 0s 29ms/step
0.8948863636363636
